In [ ]:
import datetime
import numpy as np
import pandas as pd
import pandas_profiling
import re

import plotly.express as px
%pylab inline

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 50)

In [ ]:
time_columns = ['SH_CDATE', 'GAS_ACQ_DATE', 'D_G_ACQ_DATE']
year_columns = ['WH_SPUD_YEAR']

In [ ]:
oil

In [ ]:
oil = pd.read_csv('technathon2019/challenge2/Geochemistry Data/CNS oil.csv', skiprows=[1])
rocks = pd.read_csv('technathon2019/challenge2/Geochemistry Data/CNS rock samples.csv', skiprows=[1], low_memory=False)
gas_train = pd.read_csv('technathon2019/challenge2/Geochemistry Data/CNS_gas_train.csv', skiprows=[1],
                        parse_dates=time_columns)
gas_test = pd.read_csv('technathon2019/challenge2/Geochemistry Data/CNS_gas_test.csv', skiprows=[1])

gas_train['WH_SPUD_YEAR'] = gas_train['WH_SPUD_YEAR'].fillna(1900).astype(int)

In [ ]:
gas_train[['WH_SPUD_YEAR']].profile_report()

In [ ]:
oil.iloc(1)[:2].profile_report(style={'full_width':True})

In [ ]:
report = gas_train.profile_report(style={'full_width':True})

In [ ]:
report.to_file('train_report.html')

In [ ]:
datasets = (rocks, oil, gas_train, gas_test)
dataset_names = ['rocks', 'oil', 'gas_train', 'gas_test']

In [ ]:
text_features = {}
cat_features = {}
id_columns = {}
date_columns = {}
for ds_name, ds in zip(dataset_names, datasets):
    text_features[ds_name] = [f for f, dt in ds.dtypes.items() if dt == np.dtype('O') and f not in time_columns]
    cat_features[ds_name] = [f for f, fc in ds.nunique().items() if fc < 100 and f not in text_features[ds_name]]
    id_columns[ds_name] = [c for c in ds.columns if '_ID_' in c]
    date_columns[ds_name] = [c for c in ds.columns if 'DATE' in c or c in time_columns]

In [ ]:
for ds_name, ds in zip(dataset_names, datasets):
    for c in ds.columns:
        if c in text_features[ds_name]:
            ds.loc[:, c] = ds[c].astype(str)

In [ ]:
target_columns = 'GAS_C1, GAS_C2, GAS_C3, GAS_IC4, GAS_NC4, GAS_IC5, GAS_NC5'.split(', ')

In [ ]:
feature_columns = {}
for ds_name, ds in zip(dataset_names, datasets):
    feature_columns[ds_name] = list(np.setdiff1d(ds.columns, 
                       np.concatenate([target_columns, time_columns, text_features[ds_name], 
                                       date_columns[ds_name], year_columns, id_columns[ds_name]])))

In [ ]:
X_train = gas_train[feature_columns]
X_test = gas_test[feature_columns]

y_train = gas_train[target_columns]
y_test = gas_test[target_columns]

# Catboost

In [ ]:
from catboost import CatBoostRegressor, Pool
# from catboost.eval.catboost_evaluation import *

train_dataset = Pool(data=X_train[[c for c in X_train.columns if c not in time_columns + text_features]], label=y_train   
#                , cat_features=text_features
              )
test_dataset = Pool(data=X_test[[c for c in X_test.columns if c not in time_columns + text_features]]
#                , text_features=text_features
              )

In [ ]:
cb_model = CatBoostRegressor(iterations=1000, depth=2, objective='MultiRMSE')
cb_model.fit(train_dataset, 
             verbose=100)

In [ ]:
feature_importance = pd.Series(cb_model.get_feature_importance(), 
                               index=pd.Index(train_dataset.get_feature_names(), name='feature'), name='feature_importance').sort_values()[::-1]

fig = px.bar(feature_importance.reset_index(), x='feature', y='feature_importance')
fig.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.grid_search import ParameterGrid
from sklearn.model_selection import train_test_split
from itertools import product, chain
from tqdm import tqdm

def cross_val(X, y, X_test, param, cat_features, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=500,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
#                                 eval_metric = 'Accuracy',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent'
        )
        
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        accuracy = accuracy_score(y_valid, y_pred)
        acc.append(accuracy)
    return sum(acc)/n_splits
    
def catboost_GridSearchCV(X, y, X_test, params, cat_features, n_splits=5):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, cat_features, n_splits=5)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

# Production

In [ ]:
production = pd.read_csv('technathon2019/challenge2/Production Data/CNS_Field_Production.csv',
                   parse_dates=['PERIODDATE'], dtype={'PERIODYR': int})
production['FIELDNAME'] = production['FIELDNAME'].str.lower()

In [ ]:
fields_geo = production.groupby('FIELDNAME')[['X', 'Y']].first()

In [ ]:
prod_vol = production.groupby(['FIELDNAME', 'PERIODYR'])['WATPRODVOL'].mean()

In [ ]:
plot_data = prod_vol.to_frame().join(fields_geo).reset_index().fillna(0.0).sort_values(['PERIODYR', 'FIELDNAME'])

In [ ]:
px.set_mapbox_access_token(open('.mapbox_token').read())
fig = px.scatter_mapbox(plot_data, lat='Y', lon='X', size='WATPRODVOL',
                        animation_frame='PERIODYR', animation_group='FIELDNAME', zoom=4,
                        width=512, height=512
                        )
fig.show()

In [ ]:
with open('map.html', 'w') as f:
    f.write(fig.to_html())

# Production geochemistry

In [ ]:
for ds in datasets:
    ds['FIELD_NAME'] = ds['WH_FIELD'].astype(str).str.lower()

In [ ]:
from collections import defaultdict

field_name_mapping = defaultdict(lambda: set())
field_name_mapping_inv = defaultdict(lambda: set())

for fn in np.concatenate([oil['FIELD_NAME'].unique(), rocks['FIELD_NAME'].unique(), 
                          gas_train['FIELD_NAME'].unique(), gas_test['FIELD_NAME'].unique()]):
    m = re.match('(.+)\s+\((.+)\)', fn)
    
    if m:
        well_name, field_name = m.groups()
        if field_id not in field_name_mapping:
            field_name_mapping[well_name].add(field_name)
            field_name_mapping_inv[field_name].add(well_name)

field_name_mapping = {k: v.pop() for k, v in field_name_mapping.items()}
field_name_mapping_inv = {k: v.pop() for k, v in field_name_mapping_inv.items()}

def extract_field_name(s):
    m = re.match('.+\s+\((.+)\)', s)
    if m:
        return m.groups()[0]
    elif s in field_name_mapping:
        return field_name_mapping[s]
    else:
        return s

def extract_well_name(s):
    m = re.match('(.+)\s+\(.+\)', s)
    if m:
        return m.groups()[0]
    elif s in field_name_mapping_inv:
        return field_name_mapping_inv[s]
    else:
        return s

In [ ]:
for ds in datasets:
    ds['FIELDNAME'] = ds['FIELD_NAME'].apply(extract_field_name)
    ds['WELLNAME'] = ds['FIELD_NAME'].apply(extract_well_name)

In [ ]:
prod[prod.FIELDNAME == 'cook']['PERIODYR'].unique()

In [ ]:
gas_train[gas_train.FIELDNAME == 'cook']['WH_SPUD_YEAR']

In [ ]:
oil[oil['FIELDNAME'] == 'curlew c']

In [ ]:
gas_field_years = gas_train[['FIELDNAME', 'WH_SPUD_YEAR']][
    gas_train['FIELDNAME'].isin(prod.FIELDNAME.unique()) & ~gas_train[['FIELDNAME', 'WH_SPUD_YEAR']].duplicated()] \
    .sort_values('FIELDNAME')

In [ ]:
oil_field_years = oil[['FIELDNAME', 'WH_SPUD_YEAR']][
    oil['FIELDNAME'].isin(prod.FIELDNAME.unique()) & ~oil[['FIELDNAME', 'WH_SPUD_YEAR']].duplicated()] \
    .sort_values('FIELDNAME')

In [ ]:
oil_field_years.sort_values(['FIELDNAME', 'WH_SPUD_YEAR'])

In [ ]:
# Leave latest geochemistry year
for ds in datasets:
    ds = ds.merge(ds.groupby('FIELDNAME')['WH_SPUD_YEAR'].max(skipna=True).rename('MAX_SPUD_YEAR'), on='FIELDNAME')
    ds = ds[ds['WH_SPUD_YEAR'] == ds['MAX_SPUD_YEAR']]

In [ ]:
prod_fields = ['OILPRODMBD', 'AGASPROMMS', 'WATPRODMBD']

In [ ]:
agg_fields = {c: 'sum' for c in prod_fields}
agg_fields.update({c: 'mean' for c in ['X', 'Y']})

In [ ]:
prod_performance = production[['FIELDNAME', 'PERIODYR', 'X', 'Y'] + prod_fields] \
    .groupby(['FIELDNAME', 'PERIODYR'])[['X', 'Y'] + prod_fields].agg(agg_fields) \
    .groupby('FIELDNAME')[['X', 'Y'] + prod_fields].mean()

In [ ]:
performance = {}
for ds_name, ds in zip(dataset_names, datasets):
    performance[ds_name] = prod_performance.merge(ds, how='right', on='FIELDNAME')[
        ['FIELDNAME', 'X', 'Y'] + feature_columns[ds_name] + prod_fields].dropna(subset=prod_fields)

# Gas Performance model

In [ ]:
filter_columns = ['WH_TD_M', 'WH_LONG', 'WH_LAT', 'WH_DR_ELEV_M', 'SH_DEPTH_BOT_FT', 'SH_DEPTH_TOP_FT', 'SH_FORM_BOT',]

In [ ]:
feature_columns['oil']

In [ ]:
cb_model.get_feature_importance(train_dataset, fstr_type='ShapValues')

In [ ]:
len(train_dataset.get_feature_names())

In [ ]:
from catboost import CatBoostRegressor, Pool
# from catboost.eval.catboost_evaluation import *
ds_name = 'oil'
train_dataset = Pool(data=performance[ds_name][[c for c in feature_columns[ds_name] if c not in filter_columns]], label=performance[ds_name][prod_fields]  
#                , cat_features=text_features[ds_name]
              )

cb_model = CatBoostRegressor(iterations=2000, depth=2, objective='MultiRMSE')
cb_model.fit(train_dataset, verbose=100)

feature_importance = pd.Series(cb_model.get_feature_importance(), 
                               index=pd.Index(train_dataset.get_feature_names(), name='feature'), name='feature_importance').sort_values()[::-1]

fig = px.bar(feature_importance.reset_index().iloc[:20], x='feature', y='feature_importance')
fig.show()

In [ ]:
feature_importance.index[:5]

# Anomalies

In [ ]:
anomaly_features = feature_importance.index[:5].tolist()

In [ ]:
oil_anomalies = oil[['FIELDNAME', 'WH_LAT', 'WH_LONG', 'SH_DEPTH_TOP_FT', 'SH_DEPTH_BOT_FT'] + anomaly_features] \
    .dropna(subset=['WH_LAT', 'WH_LONG', 'SH_DEPTH_TOP_FT', 'SH_DEPTH_BOT_FT'])

In [ ]:
oil_anomalies = oil_anomalies[oil_anomalies['FIELDNAME'] != 'nan']

In [ ]:
oil_anomalies['DEPTH'] = (oil_anomalies['SH_DEPTH_BOT_FT'] + oil_anomalies['SH_DEPTH_TOP_FT']) / 2

In [ ]:
oil_anomalies.to_csv('oil_anomalies.csv')

In [ ]:
oil_anomalies['FIELDNAME'].unique()

In [ ]:
data = oil_anomalies[~oil_anomalies['OIL_PRPH'].isna()][:10]